In [6]:
n0 = 0
n1 = n0 + 2500

In [2]:
import pandas as pd
import random
import numpy as np

In [3]:
authorship = pd.read_csv('G1_mentee_authorship.csv')

In [4]:
target_author = pd.read_csv('target_author.csv')['author_id'].tolist()

In [5]:
len(target_author)

5079

In [7]:
authorship.head(5)

,work_id,author_position,author_id,institution_id,first_pub_year,last_pub_year,career_len,publication_year,scientific_age
0,W100082076,first,A1998938290,NaN,1971,2021,50,2005,34
1,W100082076,last,A11059210,NaN,2000,2021,21,2005,5
2,W1009147951,first,A101861892,NaN,1918,2022,104,2000,82
3,W1009147951,last,A2167859630,NaN,1960,2011,51,2000,40
4,W101187758,first,A2096676380,I22299242,2001,2022,21,2004,3


In [8]:
len(set(authorship.work_id))

712341

In [9]:
inst_completeness = pd.read_csv('G1_mentee_inst_completeness.csv')

In [10]:
target_works1 = set(inst_completeness['work_id'][inst_completeness['ratio_inst.']>0])
len(target_works1)

553411

In [11]:
authorship = authorship[authorship.work_id.isin(target_works1)]

In [12]:
def group_info(author_i,df,backward_window,scientific_age_window):
    auship = df
    
    # Method2
    #mentor's all last-author publications
    works_i = auship['work_id'][(auship.author_id==author_i)&(auship.author_position=='last')]
    

    auship_i = auship[auship.work_id.isin(works_i)].sort_values(by='publication_year')
    # print('len(auship_i) : ',len(auship_i))

    auship_null = auship_i[~auship_i.institution_id.isnull()]

    # print('the ratio of with institution : ','{:.0%}'.format(len(auship_null)/len(auship_i)))
    
    years_i = auship_i.publication_year
    try:
        start_year = min(years_i)
        end_year = max(years_i)
    except:
        start_year = 2021
        end_year = 2022
    # print('start_year : ',start_year)
    # print('end_year : ',end_year)
    
    Group = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])
   
    for current_year in range(start_year,end_year+1):
        # print('current_year : ',current_year)
        auship_i_back = auship_i[(auship_i.publication_year>=current_year-backward_window)&
                                (auship_i.publication_year<=current_year)]
        
        auship_i_back =  auship_i_back.reset_index().drop(columns=['index'])
        #new consideration : current_scientific_age
        csa = np.array(auship_i_back['first_pub_year'] - current_year)*-1
        auship_i_back['current_scientific_age']  = csa

        auship_i_back_notnull = auship_i_back[~auship_i_back.institution_id.isnull()]
        
        
        if len(auship_i_back)==0:
            ratio_institution = 0
        else:
            ratio_institution = len(auship_i_back_notnull)/len(auship_i_back)
            
            # print('the ratio of with institution : ','{:.0%}'.format(ratio_institution))
        
        
        # ratio_institution = len(auship_i_back_notnull)/len(auship_i_back)
        # print('the ratio of with institution : ','{:.0%}'.format(ratio_institution))

        if not ratio_institution==0:
            try:
                institution_i = auship_i_back['institution_id'][auship_i_back.author_id==author_i].mode().iloc[0]
                # print('institution_i : ',institution_i)

                group_i = auship_i_back[(auship_i_back.institution_id==institution_i)
                                       &(auship_i_back.current_scientific_age<=scientific_age_window) # see above: current_scientific_age
                                       &(auship_i_back.author_id!=author_i)]

                group_size = len(set(group_i.author_id))
            # print('group_size : ',group_size)
            except:
                group_size = 0
            if group_size>0:
                group_i = group_i.assign(group_size = group_size)
                group_i = group_i.assign(mentor_id = author_i)
                group_i = group_i.assign(current_year = current_year)
                group_i = group_i.assign(ratio_institution = ratio_institution)
            else:
                group_i = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])
        else:
            group_i = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])
        Group = pd.concat((Group,group_i))
    return Group

In [13]:
backward_window = 5
scientific_age_window = 10
Data = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])

for i in range(n0,n1):
    author_i = target_author[i]
    data = group_info(author_i,authorship,backward_window,scientific_age_window)
    Data = pd.concat((Data,data))
    if i%10==0:
        print(i)

Data['current_scientific_age'] = Data['current_year'] - Data['first_pub_year']
Data.to_csv(f'groupsize/author_{n0}_{n1}.csv',index=0)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2